In [31]:
## import library 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import re

import json     
import time 
from datetime import date
import os 
import warnings
warnings.filterwarnings('ignore')

In [32]:
flatform = 'batdongsan'
today = date.today() 
name = 'huong'
order = 0

path_txt = f'd:/DUE/AI_Real_Estate/data/source/{flatform}/{today}'

try:
    os.mkdir(path_txt)
except:
    pass 

In [33]:
## attribute for selenium
opt = Options()
opt.add_argument("--disable-infobars")
opt.add_argument("start-maximized")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-notifications")
opt.add_argument("--disable-gpu")
# opt.add_argument("--headless")
opt.add_experimental_option("prefs", { \
    "profile.default_content_setting_values.media_stream_mic": 1,       
    "profile.default_content_setting_values.media_stream_camera": 1,
    "profile.default_content_setting_values.geolocation": 1, 
    "profile.default_content_setting_values.notifications": 1 
  })


In [34]:
## define function 
def create_driver(): 
    driver = webdriver.Chrome(options = opt)
    return driver

def count_page(url): 
    driver = create_driver()
    driver.get(url)
    return int(driver.find_elements(By.CLASS_NAME, 're__pagination-number')[-1].text.replace('.', ''))

def collect_item(driver, part):
    lst_url = []
    if part == 'du-an-bat-dong-san': 
        driver.execute_script("window.scrollBy(0, 100);")
        lst_items = driver.find_elements(By.CLASS_NAME, 'js__project-card.js__card-project-web.re__prj-card-full')
        for item in lst_items: 
            lst_url.append(item.find_element(By.TAG_NAME, 'a').get_attribute('href'))
    else:
        lst_items = driver.find_element(By.ID, 'product-lists-web').find_elements(By.CLASS_NAME, 'js__product-link-for-product-id')
        for item in lst_items: 
            lst_url.append(item.get_attribute('href'))
    
    return lst_url

def save_html(page_source, post_id):    
    html = BeautifulSoup(page_source, 'html5lib').find('div', class_ = 're__pr-info pr-info js__product-detail-web').prettify()
    
    name_txt = f'{path_txt}/{post_id}.txt'
    with open(name_txt, "w", encoding = 'utf-8') as f:
        f.write(html)
    return

def crawl_data(driver, news_type): 
    record = {}
    
#     post_id = driver.current_url.split('-')[-1].split('pr')[1]
    post_id = driver.current_url.split('-')[-1]
    post_id = re.sub(r'\D', '', post_id)
    
    record['post_id'] = post_id
    record['Loại tin'] = news_type
    record['Tiêu đề'] = driver.find_element(By.TAG_NAME, 'h1').text.strip() 
    record['Địa chỉ'] = driver.find_element(By.CLASS_NAME, 're__pr-short-description.js__pr-address').text.strip()
    record['Mô tả'] = driver.find_element(By.CLASS_NAME, 're__section-body.re__detail-content.js__section-body.js__pr-description.js__tracking').text.strip()
    
    keys = [i.text for i in driver.find_elements(By.CLASS_NAME, 're__pr-specs-content-item-title')]
    values = [i.text for i in driver.find_elements(By.CLASS_NAME, 're__pr-specs-content-item-value')]
    for key, value in zip(keys, values): 
        record[key.strip()] = value.strip()
    
    record['URL'] = driver.current_url
    record['date'] = str(today)
    
    tracking_id = "user-listing-ldp"
    contact = driver.find_element(By.CSS_SELECTOR, f"[tracking-id='{tracking_id}']")
    record['Liên hệ'] = contact.text.strip()
    record['card_visit'] = contact.get_attribute("href")
    
    save_html(driver.page_source, post_id)
    
    return record

def process(news_type, part, iter = None):
    lst_data = []
    
    if iter == None: 
        print('No data in crawling process!')
        return
    
    for id_page in range(1,11):  #limit page  
    # for id_page in range(1, iter):    
        driver = create_driver()
        url = f'https://batdongsan.com.vn/{part}-da-nang/p{id_page}'
        print(f'Handling on page {id_page}')
        driver.get(url)
        lst_url = collect_item(driver, part)
        driver.quit()
        
        for url in lst_url: 
            print(f'Crawling on {url}')
            
            bot = create_driver()
            bot.get(url)
            record = crawl_data(bot, news_type)
            lst_data.append(record)
            bot.quit()
    return lst_data

In [35]:
## crawling 
name_file = f'd:/DUE/AI_Real_Estate/data/record/{flatform}/{today}'
exist_data = [] 

try:
    os.mkdir(name_file)
except:
    pass 

exist_data = []     
try:
    with open(f'{name_file}/{name}_{order}.json', "r", encoding="utf-8") as file:
        exist_data = json.load(file)
except FileNotFoundError:
    pass

types_news = {'Nhà đất bán' : 'nha-dat-ban', 'Nhà đất cho thuê' : 'nha-dat-cho-thue'}
# types_news = {'Nhà đất bán' : 'nha-dat-ban'}

try: 
    for type, part in types_news.items(): 
        print(f'Crawling with {type} type')
        
        number_of_pages = count_page(f'https://batdongsan.com.vn/{part}-da-nang/p1')
        print(f'Have {number_of_pages} pages on website')
        
        exist_data += process(type, part, number_of_pages)
except Exception as e: 
    print(e)
    

# with open(f'{name_file}/{name}_{order}.json', "w", encoding="utf-8") as json_file:
#     json.dump(exist_data, json_file, ensure_ascii=False, indent=4)

Crawling with Nhà đất bán type
Message: session not created: This version of ChromeDriver only supports Chrome version 117
Current browser version is 119.0.6045.106 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe



In [36]:
exist_data

[]

In [37]:
with open(f'{name_file}/{name}_{order}.json', "w", encoding="utf-8") as json_file:
    json.dump(exist_data, json_file, ensure_ascii=False, indent=4)